In [ ]:
import 
filename = sys.argv[0]

In [ ]:
import fasttext as ft
model = ft.load_model(filename)

In [120]:
import pandas as pd
df = pd.read_csv('../data/corpus_check.csv')

In [121]:
df

,siren,legal_name,id,url_article,corpus
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,Ipsen lorgne les peptides de PeptiMimesis\n
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,Ipsen : accord important avec Probi\n \n\npubl...
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,La médecine générale d'Ipsen en panne au 1er t...
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
...,...,...,...,...,...
495,395030844,SANOFI,41863,https://www.lemondedudroit.fr/deals/57157-clif...,Clifford Chance conseil de banques sur l'émiss...
496,395030844,SANOFI,42528,https://www.lesechos.fr/industrie-services/pha...,Sanofi fragilisé par le recul de son activité ...
497,395030844,SANOFI,43721,https://www.lesechos.fr/industrie-services/pha...,Sanofi prépare son entrée sur le marché de l'a...
498,395030844,SANOFI,43756,https://www.mypharma-editions.com/sanofi-le-do...,» Sanofi : le Docteur Jacques Volckmann nommé ...


In [122]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
nlp.max_length = 68111459
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
translator = str.maketrans('', '', string.punctuation)


2020-03-31 09:04:35,832 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-03-31 09:04:35,833 - spacy_lefff.lefff - INFO - Token lefff_lemma already registered
2020-03-31 09:04:35,834 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-03-31 09:04:36,303 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [123]:
translator = str.maketrans('', '', string.punctuation)

# Tokenize text
def preprocessing(doc):
        result = []
        doc = doc.replace('\n', ' ')
        # To lowercase 
        doc = doc.lower()
        # Remove numbers
        doc = re.sub("\d+", "", doc)
        doc = doc.replace('m€', '')
        doc = doc.replace('k€', '')    
        sentences = sent_tokenize(doc)
        # Lemmentization (same as stemming but based on knowledge base)
        for sent in sentences: 
            # Delete punctuation
            sent = sent.translate(translator)
            lemmentization = []
            doc_ = nlp(sent,disable = ['ner', 'parser'])
            for d in doc_:
                lemmentization.append(d.lemma_)
            lemmentization.append('\n')
            res_ = " ".join(lemmentization)
            result.append(res_)
        return result 

def test(dataframe,index=0):
    f = open('' + str(index) + '.txt' , 'w')
    for i,row in dataframe.iterrows():
        result = "".join(preprocessing((row['text'])))
        f.write(result)  # python will convert \n to os.linesep
    f.close()  

In [124]:
#df=df.sample(10)

In [125]:
%%time
df['corpus'] = df['corpus'].map(lambda text: preprocessing(text))
df['corpus'] = df['corpus'].map(lambda text: "".join(text))
df['corpus'] = df['corpus'].map(lambda text: set(text.split()))
df['eval'] = ""


CPU times: user 38 s, sys: 7.09 ms, total: 38 s
Wall time: 38.4 s


In [127]:
# Get name of all companies in eval dataset 
vocab = set(df.legal_name)


In [128]:
from collections import defaultdict
nearest = defaultdict(str)
# Very important to use lower here, as we lower everything during preprocessing
for x in vocab:
    nearest[x.lower()] = model.get_nearest_neighbors(x.lower(),5)

In [131]:
for i,row in df.iterrows():
    nearest_words = set([y[1:len(y)-2] for x,y in (nearest[row.legal_name.lower()])])
    inters_ = nearest_words.intersection(row.corpus)
    df['eval'][i] =  inters_, len(inters_)

<ipython-input-131-64d732112e94>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eval'][i] =  inters_, len(inters_)


In [132]:
df

,siren,legal_name,id,url_article,corpus,eval
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,"{le, lorgne, peptide, ipsen, de, peptimimesi}","({ipsen}, 1)"
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,"{csp, lancement, souche, affichez, stratégique...","({ipsen}, 1)"
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,"{inscrivezvou, csp, établir, change, lancement...","({ipsen}, 1)"
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"{ligne, finance, paris, economie, cerclefinanc...","({}, 0)"
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"{ligne, finance, paris, economie, cerclefinanc...","({}, 0)"
...,...,...,...,...,...,...
495,395030844,SANOFI,41863,https://www.lemondedudroit.fr/deals/57157-clif...,"{droit, par, le, monde, sanofi, un, magazine, ...","({sanofi}, 1)"
496,395030844,SANOFI,42528,https://www.lesechos.fr/industrie-services/pha...,"{dengue, recul, difficulté, change, tcc, baiss...","({pharmaceutique, sanofi}, 2)"
497,395030844,SANOFI,43721,https://www.lesechos.fr/industrie-services/pha...,"{concurrentiel, sida, peser, différent, pertin...","({sanofi}, 1)"
498,395030844,SANOFI,43756,https://www.mypharma-editions.com/sanofi-le-do...,"{entreprise, science, américain, montpellier, ...","({pharmaceutique, sanofi}, 2)"


In [133]:
nearest

defaultdict(str,
            {'sanofi': [(0.8706918358802795, "'sanofi',"),
              (0.7755489349365234, "'pharmaceutique',"),
              (0.7721433639526367, "'pharmaceuticals',"),
              (0.7652498483657837, "'neuroendocrine',"),
              (0.7648107409477234, "'novartis',")],
             'ipsen': [(0.8306178450584412, "'ipsen',"),
              (0.766091525554657, "'spindler',"),
              (0.7527928352355957, "'medincell',"),
              (0.7455018758773804, "'astrazeneca',"),
              (0.7450808882713318, "'deinobiotics',")],
             'haulotte group': [(0.9051971435546875, "'haulotte',"),
              (0.8553677201271057, "'euromedis',"),
              (0.8337194323539734, "'schmolz+bickenbach',"),
              (0.8259358406066895, "'groupe',"),
              (0.8226117491722107, "'actelion',")],
             'spie operations': [(0.9029197692871094, "'operation',"),
              (0.8648870587348938, "'operationnel',"),
              (0.86182